In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
from plotly.colors import qualitative as pc
import yfinance as yf
import logging
import os
import json
import time
pio.renderers.default = 'notebook'

class StockResearch:
    # Aurora Borealis / Northern Lights color palette
    AURORA_COLORS = {
        'aurora_green': '#00ff87',
        'aurora_cyan': '#00ffff',
        'aurora_blue': '#4d9fff',
        'aurora_purple': '#b24dff',
        'aurora_pink': '#ff4db8',
        'aurora_violet': '#8b5cf6',
        'deep_space': '#0a0e27',
        'midnight_blue': '#1a1f3a',
        'star_white': '#e0f2ff'
    }
    
    AURORA_GRADIENTS = [
        ['#00ff87', '#00ffff', '#4d9fff'],  # Green to Cyan to Blue
        ['#4d9fff', '#b24dff', '#ff4db8'],  # Blue to Purple to Pink
        ['#00ffff', '#b24dff', '#00ff87'],  # Cyan to Purple to Green
        ['#8b5cf6', '#ff4db8', '#00ff87'],  # Violet to Pink to Green
        ['#00ff87', '#4d9fff', '#b24dff'],  # Green to Blue to Purple
    ]
    
    def __init__(self, file=None, configs=None):
        self.logger = logging.getLogger(__name__)
        self.file = file
        self.configs = configs
        self.stocks_existing, configs_existing = self._load_stocks(), self._load_configs()
        if self.stocks_existing:
            if not configs_existing:
                self.interval = '1wk'
                self.freq = 'quarterly'
            self._get_data()
    
    def _load_stocks(self):
        if self.file is None:
            return False
        try:
            with open(self.file, 'r') as f:
                self.tickers = [i.strip() for i in f.readlines()]
                return bool(self.tickers)
        except FileNotFoundError:
            self.logger.error(f'{self.file} not found in cache')
            return False
    
    def _load_configs(self):
        if self.configs is None:
            return False
        try:
            with open(self.configs, 'r') as f:
                content = json.load(f)
                try:
                    self.interval = content['interval']
                    self.freq = content['freq']
                    return True
                except KeyError:
                    self.logger.error('Configs is missing required data')
                    return False
        except FileNotFoundError:
            self.logger.warning('Configs file not found in cache')
            return False
    
    def _get_data(self):
        for ticker in self.tickers:
            if not os.path.exists(f'{ticker}_fundamental_data_{self.freq}.csv'):
                self._get_fundamentals(ticker)
                time.sleep(0.5)
            if not os.path.exists(f'{ticker}_price_data_{self.interval}.csv'):
                self._get_ohlc(ticker)
                time.sleep(0.5)
    
    def _get_fundamentals(self, ticker):
        try:
            obj = yf.Ticker(ticker=ticker)
            balance = obj.get_balancesheet(freq=self.freq)
            income = obj.get_income_stmt(freq=self.freq)
            cashflow = obj.get_cashflow(freq=self.freq)
            df = pd.concat([income, cashflow], join='inner', axis=0)
            df = df[df.columns[::-1]]
            if self.freq == 'quarterly':
                df.columns = df.columns.to_period('Q').astype('str')
            else:
                df.columns = df.columns.year.astype('str')
            df.to_csv(f'{ticker}_fundamental_data_{self.freq}.csv')
        except Exception as e:
            self.logger.warning(f'Could not fetch Income-Statement for {ticker}: {e}')
    
    def _get_ohlc(self, ticker):
        try:
            df = yf.download(ticker, interval=self.interval, period='max', auto_adjust=True, progress=False)
            df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            df.to_csv(f'{ticker}_price_data_{self.interval}.csv')
        except Exception as e:
            self.logger.warning(f'Could not fetch OHLC for {ticker}: {e}')
    
    def _get_aurora_layout(self, title, subtitle=None):
        """Aurora Borealis inspired layout with flowing gradient backgrounds"""
        layout = dict(
            font=dict(size=13, color='#e0f2ff', family='Optima, Segoe UI, sans-serif'),
            paper_bgcolor='#0a0e27',
            plot_bgcolor='rgba(26, 31, 58, 0.4)',
            title=dict(
                text=f'<b style="background: linear-gradient(90deg, #00ff87, #00ffff, #4d9fff, #b24dff); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{title}</b>' + 
                     (f'<br><sub style="color:#4d9fff; font-size:15px; opacity:0.9;">{subtitle}</sub>' if subtitle else ''),
                font=dict(size=28, weight='bold'),
                x=0.5,
                xanchor='center',
                y=0.95,
                yanchor='top'
            ),
            margin=dict(l=75, r=75, t=115, b=75),
            hoverlabel=dict(
                bgcolor='rgba(10, 14, 39, 0.95)',
                font=dict(size=13, color='#00ff87', family='Segoe UI, sans-serif'),
                bordercolor='#00ffff',
                namelength=-1
            ),
            transition=dict(duration=800, easing='cubic-in-out'),
            # Soft flowing grid like northern lights
            xaxis=dict(
                showgrid=True,
                gridcolor='rgba(0, 255, 255, 0.08)',
                gridwidth=1.5,
                zeroline=False
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='rgba(0, 255, 135, 0.1)',
                gridwidth=1.5,
                zeroline=True,
                zerolinecolor='rgba(77, 159, 255, 0.3)',
                zerolinewidth=2
            )
        )
        return layout
    
    def price_to_earnings(self):
        if self.stocks_existing:
            pe_df = pd.DataFrame()
            for ticker in self.tickers:
                try:
                    df = pd.read_csv(f'{ticker}_fundamental_data_{self.freq}.csv', index_col=0)
                    if self.freq == 'yearly':
                        diluted_eps = df.loc['DilutedEPS'].dropna().iloc[-1]
                    else:
                        if len(df.loc['DilutedEPS'].dropna()) < 4:
                            fill = (4 - len(df.loc['DilutedEPS'].dropna())) * df.loc['DilutedEPS'].dropna().mean()
                            diluted_eps = df.loc['DilutedEPS'].dropna().sum() + fill
                        else:
                            diluted_eps = df.loc['DilutedEPS'].dropna().iloc[-4:].sum()
                    
                    df = pd.read_csv(f'{ticker}_price_data_{self.interval}.csv', index_col=0)
                    price = df['Close'].iloc[-1]
                    pe = price / diluted_eps
                    if pe < 0:
                        pe = np.NaN
                    pe_df.loc[ticker, 'PE_ttm'] = round(pe, 2)
                except Exception as e:
                    pe = np.NaN
                    pe_df.loc[ticker, 'PE_ttm'] = pe
                    self.logger.warning(f'Diluted EPS of {ticker} not available')
                
                try:
                    time.sleep(0.5)
                    obj = yf.Ticker(ticker=ticker)
                    fpe = obj.info['forwardPE']
                    pe_df.loc[ticker, 'Forward_PE'] = round(fpe, 2)
                except Exception as e:
                    fpe = np.NaN
                    pe_df.loc[ticker, 'Forward_PE'] = fpe
                    self.logger.warning(f'No forward EPS for {ticker} available')
                
                discount = (pe / fpe - 1) * 100 if (pe and fpe and fpe != 0) else np.NaN
                pe_df.loc[ticker, 'Difference(%)'] = round(discount, 2)
            
            if not pe_df.empty:
                pe_df.sort_values(by='Difference(%)', ascending=False, inplace=True)
                return list(pe_df.dropna().index[:int(len(pe_df) * 0.33)])
            else:
                return []
    
    def return_correlation(self, stocks=None):
        if self.stocks_existing:
            tickers = stocks if stocks else self.tickers
            
            to_merge = []
            for ticker in tickers:
                try:
                    df = pd.read_csv(f'{ticker}_price_data_{self.interval}.csv', index_col=0, parse_dates=True)
                    to_merge.append(df['Close'].rename(ticker))
                except FileNotFoundError:
                    continue
            
            merged_df = pd.concat(to_merge, join='inner', axis=1)
            corr = merged_df.pct_change(axis=0).corr()
            
            # Aurora heatmap with flowing colors
            fig = go.Figure()
            fig.add_trace(
                go.Heatmap(
                    z=corr.values,
                    x=corr.columns,
                    y=corr.columns,
                    zmid=0,
                    colorscale=[
                        [0, '#0a0e27'], [0.15, '#1a1f3a'], [0.3, '#2d1b69'],
                        [0.45, '#4d1380'], [0.5, '#1a1f3a'],
                        [0.55, '#004d4d'], [0.7, '#00ff87'], [0.85, '#00ffff'], [1, '#4d9fff']
                    ],
                    colorbar=dict(
                        title=dict(text='<b>ρ</b>', side='right', font=dict(size=18, color='#00ffff')),
                        tickfont=dict(size=11, color='#e0f2ff'),
                        len=0.75,
                        thickness=18,
                        outlinecolor='#00ff87',
                        outlinewidth=2,
                        bgcolor='rgba(10, 14, 39, 0.7)'
                    ),
                    hovertemplate='<b>%{x}</b> ✦ <b>%{y}</b><br>Correlation: %{z:.3f}<extra></extra>',
                    text=corr.values,
                    texttemplate='%{text:.2f}',
                    textfont=dict(size=10, color='rgba(224, 242, 255, 0.9)'),
                    showscale=True
                )
            )
            
            layout = self._get_aurora_layout('✧ Return Correlation Matrix ✧', 'Cross-Asset Synchronization')
            layout['xaxis'].update(
                tickangle=-45,
                tickfont=dict(size=11, color='#00ffff'),
                showgrid=False
            )
            layout['yaxis'].update(
                tickfont=dict(size=11, color='#00ffff'),
                showgrid=False
            )
            layout['height'] = 750
            fig.update_layout(**layout)
            fig.show()
        return True
    
    def growth(self, stocks=None):
        if self.stocks_existing:
            tickers = stocks if stocks else self.tickers
            
            for ticker in tickers:
                try:
                    df = pd.read_csv(f'{ticker}_fundamental_data_{self.freq}.csv', index_col=0)
                except FileNotFoundError:
                    continue
                
                try:
                    required = df.loc[['NetIncome', 'FreeCashFlow', 'TotalRevenue']].dropna(axis=1)
                    netmargin = required.loc['NetIncome'] / required.loc['TotalRevenue'] * 100
                    fcfmargin = required.loc['FreeCashFlow'] / required.loc['TotalRevenue'] * 100
                    revgrowth = required.loc['TotalRevenue'].pct_change().fillna(0.0) * 100
                    margins = {'FCF Margin': fcfmargin, 'Net Margin': netmargin}
                except KeyError:
                    self.logger.warning(f'{ticker} has not sufficient data')
                    continue
                
                fig = go.Figure()
                
                # Aurora-colored bars with shimmer effect
                bar_colors = ['#00ff87', '#4d9fff']
                for j, (name, values) in enumerate(margins.items()):
                    fig.add_trace(
                        go.Bar(
                            x=values.index,
                            y=values,
                            name=f'<b>{name}</b>',
                            marker=dict(
                                color=bar_colors[j],
                                line=dict(color='rgba(224, 242, 255, 0.4)', width=2.5),
                                opacity=0.85
                            ),
                            hovertemplate='<b>%{x}</b><br>' + name + ': <b>%{y:.1f}%</b><extra></extra>',
                            showlegend=True
                        )
                    )
                
                # Flowing aurora growth line
                fig.add_trace(
                    go.Scatter(
                        x=revgrowth.index,
                        y=revgrowth,
                        name='<b>Revenue Growth</b>',
                        mode='lines+markers',
                        line=dict(
                            color='#b24dff',
                            width=5,
                            shape='spline'
                        ),
                        marker=dict(
                            color='#ff4db8',
                            symbol='circle',
                            size=15,
                            line=dict(color='#e0f2ff', width=3),
                            opacity=0.9
                        ),
                        hovertemplate='<b>%{x}</b><br>Growth: <b>%{y:.1f}%</b><extra></extra>',
                        yaxis='y2'
                    )
                )
                
                layout = self._get_aurora_layout(f'✦ {ticker} Profitability & Momentum ✦', 'Financial Performance Metrics')
                layout.update(
                    xaxis=dict(
                        tickfont=dict(size=11, color='#b8d4ff'),
                        title=dict(text='<b>Period</b>', font=dict(size=12, color='#00ffff'))
                    ),
                    yaxis=dict(
                        title=dict(text='<b>Margin (%)</b>', font=dict(size=12, color='#00ff87')),
                        tickfont=dict(color='#00ff87'),
                        gridcolor='rgba(0, 255, 135, 0.12)'
                    ),
                    yaxis2=dict(
                        title=dict(text='<b>Growth (%)</b>', font=dict(size=12, color='#b24dff')),
                        overlaying='y',
                        side='right',
                        tickfont=dict(color='#ff4db8'),
                        gridcolor='rgba(178, 77, 255, 0.1)'
                    ),
                    barmode='group',
                    legend=dict(
                        orientation='h',
                        yanchor='bottom',
                        y=1.02,
                        xanchor='center',
                        x=0.5,
                        bgcolor='rgba(26, 31, 58, 0.85)',
                        bordercolor='#00ffff',
                        borderwidth=2,
                        font=dict(size=11, color='#e0f2ff')
                    ),
                    height=620,
                    bargap=0.2
                )
                fig.update_layout(**layout)
                fig.show()
    
    def risk_and_return(self, startdate=None, stocks=None):
        if self.stocks_existing:
            tickers = stocks if stocks else self.tickers
            
            if startdate is None:
                self.logger.warning('Stats are calculated with different startpoints for each stock')
            else:
                try:
                    startdate = pd.to_datetime(startdate)
                except Exception as e:
                    self.logger.error('Invalid Startdate')
                    return
            
            to_merge = []
            stats = pd.DataFrame()
            
            for ticker in tickers:
                try:
                    df = pd.read_csv(f'{ticker}_price_data_{self.interval}.csv', index_col=0, parse_dates=True)['Close']
                    if startdate is not None:
                        df = df.loc[df.index >= startdate]
                except FileNotFoundError:
                    continue
                
                to_merge.append(df.rename(ticker))
                
                annualize_map = {'1d': 252, '1wk': 52, '1mo': 12}
                annualize = annualize_map.get(self.interval)
                
                returns = df.pct_change().dropna() * 100
                if annualize is not None:
                    fixed_rfr = 4
                    mean = returns.mean() * annualize
                    std = returns.std() * np.sqrt(annualize)
                    sr = (mean - fixed_rfr) / std
                    
                    stats.loc[ticker, 'Average_Return'] = round(mean, 2)
                    stats.loc[ticker, 'Return_Volatility'] = round(std, 2)
                    stats.loc[ticker, 'Sharpe_Ratio'] = round(sr, 2)
            
            if annualize is not None:
                allstocks_df = pd.concat(to_merge, join='inner', axis=1)
                allstocksreturn = (allstocks_df.pct_change() * 100).dropna().mean(axis=1)
                allstocks_mean = allstocksreturn.mean() * annualize
                allstocks_std = allstocksreturn.std() * np.sqrt(annualize)
                allstocks_sr = (allstocks_mean - fixed_rfr) / allstocks_std
                
                stats.loc['✦ Mean ✦', 'Average_Return'] = round(allstocks_mean, 2)
                stats.loc['✦ Mean ✦', 'Return_Volatility'] = round(allstocks_std, 2)
                stats.loc['✦ Mean ✦', 'Sharpe_Ratio'] = round(allstocks_sr, 2)
            
            if not stats.empty:
                print(stats.sort_values(by='Sharpe_Ratio', ascending=False))
            
            # Aurora flowing area chart
            returns = (allstocks_df.mean(axis=1).iloc[1:] / allstocks_df.iloc[0].mean() - 1) * 100
            
            fig = go.Figure()
            
            # Multi-layer aurora glow effect
            for i, opacity in enumerate([0.15, 0.25, 0.35]):
                fig.add_trace(
                    go.Scatter(
                        x=allstocks_df.index[1:],
                        y=returns,
                        mode='lines',
                        line=dict(color='rgba(0, 255, 255, 0)', width=0),
                        fill='tozeroy',
                        fillcolor=f'rgba(0, 255, 135, {opacity})',
                        showlegend=False,
                        hoverinfo='skip'
                    )
                )
            
            # Main aurora line with gradient
            fig.add_trace(
                go.Scatter(
                    x=allstocks_df.index[1:],
                    y=returns,
                    mode='lines',
                    line=dict(
                        color='#00ffff',
                        width=4,
                        shape='spline'
                    ),
                    name='<b>Portfolio Return</b>',
                    hovertemplate='<b>%{x|%Y-%m-%d}</b><br>Return: <b>%{y:.2f}%</b><extra></extra>',
                    fill='tonexty',
                    fillgradient=dict(
                        type='vertical',
                        colorscale=[
                            [0, 'rgba(0, 255, 135, 0.4)'],
                            [0.5, 'rgba(77, 159, 255, 0.3)'],
                            [1, 'rgba(178, 77, 255, 0.2)']
                        ]
                    )
                )
            )
            
            # Horizon reference line
            fig.add_hline(
                y=0,
                line_dash='dot',
                line_color='rgba(77, 159, 255, 0.5)',
                line_width=2.5,
                annotation_text='✦ Break Even ✦',
                annotation_font=dict(size=11, color='#4d9fff'),
                annotation_position='right'
            )
            
            layout = self._get_aurora_layout('✧ Historic Equal-Weight Return ✧', 'Portfolio Performance Timeline')
            layout.update(
                xaxis=dict(
                    tickfont=dict(size=11, color='#b8d4ff'),
                    rangeslider=dict(
                        visible=True,
                        bgcolor='rgba(26, 31, 58, 0.6)',
                        bordercolor='#00ff87',
                        borderwidth=2
                    )
                ),
                yaxis=dict(
                    title=dict(text='<b>Return (%)</b>', font=dict(size=13, color='#00ffff')),
                    tickfont=dict(color='#00ffff'),
                    gridcolor='rgba(0, 255, 255, 0.1)'
                ),
                height=650
            )
            fig.update_layout(**layout)
            fig.show()
            
            if annualize is None:
                self.logger.warning('Stats only available for [1d, 1wk, 1mo]')
            return True
    
    def diversication(self, stocks=None):
        if self.stocks_existing:
            tickers = stocks if stocks else self.tickers
            
            div_df = pd.Series(dtype='float')
            
            for ticker in tickers:
                stock = yf.Ticker(ticker)
                industry = stock.info.get('industry', 'Unknown')
                div_df.loc[ticker] = industry
                time.sleep(0.5)
            
            n = div_df.value_counts()
            
            # Aurora pie with flowing colors
            fig = go.Figure()
            
            # Northern lights color palette
            colors = ['#00ff87', '#00ffff', '#4d9fff', '#b24dff', '#ff4db8', '#8b5cf6', '#06ffa5', '#4dd2ff']
            colors = colors[:len(n)]
            
            fig.add_trace(
                go.Pie(
                    values=n.values,
                    labels=n.index,
                    marker=dict(
                        colors=colors,
                        line=dict(color='#0a0e27', width=4)
                    ),
                    hole=0.55,
                    textinfo='label+percent',
                    textfont=dict(size=12, color='#0a0e27', family='Segoe UI', weight='bold'),
                    hovertemplate='<b>%{label}</b><br>Count: %{value}<br>Share: %{percent}<extra></extra>',
                    pull=[0.06] * len(n),
                    rotation=45,
                    sort=False
                )
            )
            
            # Aurora center with shimmer
            fig.add_annotation(
                text=f'<b style="color:#00ffff; font-size:36px;">{len(tickers)}</b><br>' +
                     f'<span style="color:#00ff87; font-size:15px;">Stocks</span><br>' +
                     f'<span style="color:#b24dff; font-size:13px;">{len(n)} Sectors</span>',
                font=dict(family='Segoe UI'),
                showarrow=False,
                x=0.5,
                y=0.5
            )
            
            layout = self._get_aurora_layout('✦ Portfolio Diversification ✦', 'Sector Allocation Overview')
            layout.update(
                showlegend=True,
                legend=dict(
                    orientation='v',
                    yanchor='middle',
                    y=0.5,
                    xanchor='left',
                    x=1.05,
                    bgcolor='rgba(26, 31, 58, 0.85)',
                    bordercolor='#00ff87',
                    borderwidth=2,
                    font=dict(size=11, color='#e0f2ff')
                ),
                height=700
            )
            layout['xaxis']['showgrid'] = False
            layout['yaxis']['showgrid'] = False
            fig.update_layout(**layout)
            fig.show()
    
    def __call__(self):
        dc = self.price_to_earnings()
        self.diversication(dc)
        self.return_correlation(dc)
        self.risk_and_return('2020-01-01', dc)
        self.growth(dc)


if __name__ == '__main__':
    logging.basicConfig(level=logging.WARNING)
    obj = StockResearch('DJI.txt', 'configs2.json')
    obj()